# Datenvorbereitung für ML-Modelle (Umsatzprojekt)

Dieses Notebook erstellt alle relevanten Features für die spätere Modellierung.  
Es erzeugt Trainings-, Validierungs- und Testdaten und speichert sie als `.pkl`,  
um sie wie im Kursbeispiel wiederverwenden zu können.

In [1]:
import pandas as pd
import numpy as np
from itertools import product
from workalendar.europe import Germany
from sklearn.model_selection import train_test_split
import os

# Datenquellen
base = "https://raw.githubusercontent.com/opencampus-sh/einfuehrung-in-data-science-und-ml/main/"
df_umsatz = pd.read_csv(base + "umsatzdaten_gekuerzt.csv")
df_wetter = pd.read_csv(base + "wetter.csv")
df_kiwo = pd.read_csv(base + "kiwo.csv")
df_ferien = pd.read_csv("../../data/ferien_sh.csv", parse_dates=["Startdatum", "Enddatum"])

In [2]:
# Vollständiger Grid
alle_daten = pd.date_range("2013-07-01", "2019-07-31")
warengruppen = [1, 2, 3, 4, 5, 6]
voll_kombis = pd.DataFrame(list(product(alle_daten, warengruppen)), columns=["Datum", "Warengruppe"])

# Wetter
df_wetter["Datum"] = pd.to_datetime(df_wetter["Datum"])
df = voll_kombis.merge(df_wetter, on="Datum", how="left")

# KiWo
df_kiwo["Datum"] = pd.to_datetime(df_kiwo["Datum"])
df_kiwo["KielerWoche"] = 1
df = df.merge(df_kiwo[["Datum", "KielerWoche"]], on="Datum", how="left").fillna({"KielerWoche": 0})

# Umsatz
df_umsatz["Datum"] = pd.to_datetime(df_umsatz["Datum"])
df = df.merge(df_umsatz, on=["Datum", "Warengruppe"], how="left")
df["Umsatz"] = df["Umsatz"].fillna(0)

In [3]:
df["Wochentag"] = df["Datum"].dt.dayofweek
df["Monat"] = df["Datum"].dt.month
df["IstWochenende"] = df["Wochentag"].isin([5, 6]).astype(int)

# Feiertage
cal = Germany(subdivision="SH")
feiertage = set()
for jahr in df["Datum"].dt.year.unique():
    feiertage.update([d for d, _ in cal.holidays(jahr)])
df["Feiertag"] = df["Datum"].isin(feiertage).astype(int)

# Ferien
ferien_tage = set()
for _, row in df_ferien.iterrows():
    ferien_tage.update(pd.date_range(row["Startdatum"], row["Enddatum"]))
df["Ferienzeit"] = df["Datum"].isin(ferien_tage).astype(int)

/var/folders/09/wpbg8c7x22x3d_2_ybk_s1g40000gn/T/ipykernel_60946/1497446009.py:10: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  df["Feiertag"] = df["Datum"].isin(feiertage).astype(int)


In [4]:
df["Temperatur"] = df["Temperatur"].fillna(df["Temperatur"].median())
df["Bewoelkung"] = df["Bewoelkung"].fillna(df["Bewoelkung"].median())
df["Windgeschwindigkeit"] = df["Windgeschwindigkeit"].fillna(df["Windgeschwindigkeit"].median())
df["Wettercode"] = df["Wettercode"].fillna(-1).astype(int)

# Features & Target
features = ["Warengruppe", "Temperatur", "Bewoelkung", "Windgeschwindigkeit",
            "KielerWoche", "Wochentag", "Monat", "IstWochenende", "Feiertag", "Ferienzeit"]
target = "Umsatz"

In [ ]:
X = df[features]
y = df[[target]]

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.33, random_state=42)  # 0.33 ≈ 0.10

# Pickle exportieren
pickle_dir = "../data/pickle_data"
os.makedirs(pickle_dir, exist_ok=True)

X_train.to_pickle("pickle_data/training_features.pkl")
y_train.to_pickle("pickle_data/training_labels.pkl")
X_val.to_pickle("pickle_data/validation_features.pkl")
y_val.to_pickle("pickle_data/validation_labels.pkl")
X_test.to_pickle("pickle_data/test_features.pkl")
y_test.to_pickle("pickle_data/test_labels.pkl")

print("✅ Alle Pickles gespeichert in pickle_data/")

OSError: Cannot save file into a non-existent directory: 'pickle_data'